In [ ]:
from selenium import webdriver
import os
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support.select import Select
import lxml.html
from selenium.common.exceptions import NoSuchElementException
import requests
from bs4 import BeautifulSoup
import pandas as pd
from nltk.stem import WordNetLemmatizer 
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [ ]:
driverLocation= "C:\\Users\\User\\Desktop\\chromedriver.exe"
os.environ['webdriver.chrome.drive'] = driverLocation

In [ ]:
News_Headline=[]
News_Date=[]

driver = webdriver.Chrome(driverLocation)
for i in range(1,1094):
    driver.get('https://www.investing.com/news/cryptocurrency-news/{}'.format(i))
    time.sleep(2)
    pagesource = driver.page_source
    soup=BeautifulSoup(pagesource,'html.parser') 
    Article=soup.find_all("div",{'class':'textDiv'})
    for item in range(len(Article)):
        if "<p>" not in str(Article[item]) or "date" not in str(Article[item]):
            pass
        else:
            News_Headline.append(Article[item].a.attrs['title'])
            News_Date.append(Article[item].find_all('span',{'class':'date'})[0].text.replace(u'\xa0', u'').replace('-','').replace(',',''))
    print('progress {}/1094'.format(i))

In [ ]:
analyser = SentimentIntensityAnalyzer()

In [ ]:
def sentiment_analyzer_scores(sentence):
    return analyser.polarity_scores(sentence)

In [ ]:
lemmatizer = WordNetLemmatizer() 

In [ ]:
def lemmatizer_analyzer_lemattize(sentence):
    return lemmatizer.lemmatize(sentence)

In [ ]:
import pandas as pd
df=pd.DataFrame(list(zip(News_Date,News_Headline)),
              columns=['Date','Headline'])

In [ ]:
df = df.applymap(lambda s:s.lower() if type(s) == str else s)

In [ ]:
import re
from textblob import TextBlob
import matplotlib.pyplot as plt
import os
import glob 
import numpy as np

def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
        
    return input_txt

In [ ]:
df['tidy_Text'] = np.vectorize(remove_pattern)(df['Headline'], "@[\w]*")

In [ ]:
df['tidy_Text'] = df['tidy_Text'].str.replace("[^a-zA-Z#]", " ")

In [ ]:
from string import punctuation
def strip_punctuation(s):
    return ''.join(c for c in s if c not in punctuation)

df["tidy_Text"] = df["tidy_Text"].apply(strip_punctuation)

In [ ]:
def remove_text(s):
    return re.sub(r'\b\w{1,3}\b', '', s)

In [ ]:
df["tidy_Text"] = df["tidy_Text"].apply(remove_text)

In [ ]:
df['tidy_Text'] = df['tidy_Text'].apply(lemmatizer_analyzer_lemattize)

In [ ]:
def detect_polarity(text):
    return TextBlob(text).sentiment.polarity

df["polarity"] = df["tidy_Text"].apply(detect_polarity)

In [ ]:
new_list=df["tidy_Text"].apply(sentiment_analyzer_scores)

In [ ]:
neg=[]
neu=[]
pos=[]
compound=[]

for dic in new_list:
    for val,cal in dic.items():
        if val=='neg':
            neg.append(cal)
        elif val=='neu':
            neu.append(cal)
        elif val=='pos':
            pos.append(cal)
        else:
            compound.append(cal)

In [ ]:
df["Vader_neg"]=neg
df["Vader_neu"]=neu
df["Vader_pos"]=pos
df["Vader_compound"]=compound

In [ ]:
df.tail(50)

In [ ]:
df_textless=df.drop(['Headline','tidy_Text'],axis=1)

In [ ]:
df_textless.head()

In [ ]:
c1=df_textless['polarity']!=0
c2=df_textless['Vader_neu']!=1
c3=df_textless['Vader_compound']!=0

In [ ]:
df_textless[c1&c2&c3]

In [ ]:
dategroup = df_textless.groupby('Date')

In [ ]:
df_output=dategroup.mean()

In [ ]:
df_output.to_csv("Bitcoin_News_Sen_Value.csv", encoding='utf-8', index=True)